In [132]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import os
from numpy.linalg import norm
from matching.games import HospitalResident

In [3]:
data_EHR1 = pd.read_csv("./data_EHR1.csv")
data_EHR1.head()

,f1_1,f2_1,f3_1,f4_1,f5_1,f6_1,f7_1,f8_1,f9_1,f10_1,f11_1,f12_1,f13_1,f14_1,f15_1,f16_1,f17_1,f18_1,f19_1,f20_1
0,1.696794,-0.621645,-2.350322,0.857568,0.183480,0.929362,1.662710,3.002824,-0.952821,0.780778,-2.149981,-0.096125,0.686564,0.932175,1.035598,0.100794,0.472664,0.961984,-1.581007,0.418345
1,4.490176,-1.369651,-6.381362,2.446793,0.648104,5.127069,3.632748,6.645584,-1.694836,1.961207,-5.393690,-1.725572,1.115977,2.599284,1.849126,0.149453,1.651858,2.380088,-2.989520,1.523643
2,6.195791,-1.803062,-8.885239,1.678346,1.484633,7.669556,5.218378,9.897765,-2.370407,2.332558,-7.979431,-2.123378,1.262524,3.922189,2.477547,1.246901,1.795737,2.639566,-4.948484,1.801274
3,5.943051,0.117324,-11.007327,0.045042,2.026570,7.370177,6.339163,11.226333,-2.778230,3.804610,-8.785526,-1.515712,0.418063,2.784484,2.132019,1.920771,1.803917,2.866873,-4.924506,1.716181
4,1.792690,2.802358,-5.212722,-1.270905,0.897933,4.450223,2.796456,7.543190,-1.245164,1.265860,-4.883092,-0.436753,-0.691122,2.223674,-0.473904,0.735597,1.970611,0.547733,-3.001847,0.299120


In [7]:
data_EHR1.shape

(60000, 20)

In [4]:
data_EHR2 = pd.read_csv("./data_EHR2.csv")
data_EHR2.head()

,f1_2,f2_2,f3_2,f4_2,f5_2,f15_2,f14_2,f6_2,f17_2,f9_2,f7_2,f12_2,f20_2,f19_2,f11_2,f18_2,f10_2,f16_2,f8_2,f13_2
0,1.002464,-0.054770,-1.607723,0.413153,0.526490,0.808553,0.521133,1.489252,1.093870,-0.143866,1.521181,-1.214374,-0.054487,-0.620618,-0.165420,0.877329,0.684882,-0.041809,2.285389,-0.688339
1,-0.208255,-0.430312,3.542310,0.273496,0.523185,1.403259,0.807507,-0.765867,0.872294,0.744647,-0.662414,0.211199,-0.622928,0.033095,1.369378,1.721680,-1.527514,-0.234272,0.200160,-0.147486
2,-3.177253,-0.359707,7.928680,0.673136,-0.539660,0.083743,0.467750,-3.756303,-0.516528,1.122243,-4.087978,-0.085167,-1.435108,-0.203887,3.237390,1.130163,-3.310383,-1.228359,-3.529958,0.776517
3,-3.293687,-0.708697,6.471826,0.595019,-2.137115,-0.089151,-1.102489,-3.336020,-0.365526,0.826688,-4.684949,-0.328677,-1.541318,-0.260947,2.497054,-0.693162,-2.074722,-1.027206,-3.263632,0.990452
4,0.941395,-1.305723,3.126918,-0.427142,-2.480852,-0.253852,-1.456013,-1.420847,0.261801,0.014554,-1.940401,-0.648343,-1.986704,-0.346978,-0.000081,-1.558701,-0.040872,-0.748870,-0.606107,1.572000


In [8]:
data_EHR2.shape

(60000, 20)

# Get golden-standard-list from permutation matrix for the unmapped features in 2 EHRs

In [5]:
# get permutation matrix for the unmapped features in 2 EHRs
ump_f_EHR1 = list(data_EHR1.columns[5:])
ump_f_EHR2 = list(data_EHR2.columns[5:])

print(ump_f_EHR1)
print(ump_f_EHR2)

['f6_1', 'f7_1', 'f8_1', 'f9_1', 'f10_1', 'f11_1', 'f12_1', 'f13_1', 'f14_1', 'f15_1', 'f16_1', 'f17_1', 'f18_1', 'f19_1', 'f20_1']
['f15_2', 'f14_2', 'f6_2', 'f17_2', 'f9_2', 'f7_2', 'f12_2', 'f20_2', 'f19_2', 'f11_2', 'f18_2', 'f10_2', 'f16_2', 'f8_2', 'f13_2']


In [6]:
p = np.zeros((len(ump_f_EHR1), len(ump_f_EHR2)))
for i in range(len(ump_f_EHR1)):
    for j in range(len(ump_f_EHR2)):
        if ump_f_EHR1[i][1:-2] == ump_f_EHR2[j][1:-2]:
            p[i][j] = 1
p

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

# Training seq-to-seq model to regard transformation function as the fingerprint

In [11]:
# transfer gen_data shape from (60000, 20) to (2500, 24, 20)
data_1 = data_EHR1.to_numpy().reshape((2500, 24, 20))
data_2 = data_EHR2.to_numpy().reshape((2500, 24, 20))

### Define Dataset
* input: unmapped feature i in 2 EHRS (totally 15 unmapped features in 2 EHRs, the last 15 columns in data_1/data_2)
* target: pre-mapped features in 2 EHRS (the first 5 columns)


In [39]:
class Seq2seq_Dataset(Dataset):
    def __init__(self, data, EHR_version): 
        self.patients_num = data.shape[0]
        self.ump_features_list = []
        self.mp_features_list = []
        
        # load mp_features
        if os.path.exists("./mp_f_tensors_" + str(EHR_version) + ".pt"):
            self.mp_features_list = torch.load("./mp_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish loading mapped features' tensors!")
        else:
            for i in range(self.patients_num):
                cur_mp_features = torch.tensor(data[i,:,:5])
                self.mp_features_list.append(cur_mp_features.float())
            
            torch.save(self.mp_features_list, "./mp_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish transforming mapped features' tensors!")
        
        # load ump_features
        if os.path.exists("./ump_f_tensors_" + str(EHR_version) + ".pt"):
            self.ump_features_list = torch.load("./ump_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish loading unmapped features' tensors!")
        else:
            for i in range(self.patients_num):
                cur_ump_features = torch.tensor(data[i, :, 5:])
                self.ump_features_list.append(cur_ump_features.float())
                
            torch.save(self.ump_features_list, "./ump_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish transforming unmapped features' tensors!")
            
    def __len__(self):
        return self.patients_num
    
    def __getitem__(self, idx):
        sample_mp_features = self.mp_features_list[idx]
        # shape: [24, 5]
        sample_ump_features = self.ump_features_list[idx]
        # shape: [24, 15]
        return sample_mp_features, sample_ump_features
        
        

# Define model_1: feed final hidden state(on the last time stamp of each rnn layer (in our case, the # of rnn layer is only 1), with shape: [1, batch_size, hidden_dim]) to MLP to predict mapped time series data

In [15]:
import torch.nn as nn
from torch import optim
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import average_precision_score, roc_auc_score
import random
# from torch.nn import BCEWithLogitsLoss

In [37]:
class Seq2seq_model(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(Seq2seq_model, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True, dropout=0.2)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        self.target_dim = seq_len * num_mp_f
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.MLP_drop1 = nn.Dropout(p=0.2)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim / 2))
        self.MLP_drop2 = nn.Dropout(p=0.2)
        self.dense3 = nn.Linear(int(self.target_dim / 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = nn.Flatten()(input_data[0]) 
        # shape: [10, 24*5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, h_n = self.RNN(unmapped_features, self.h_0)
        # h_n: final hidden state with shape [1, batch_size, hidden_dim]
        map_predict = self.dense1(h_n[0])
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop1(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop2(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        
        
        

# K-Fold Cross Validation

### a. Define train(), val() to call in each epoch

In [89]:
def train(train_loader, model, optimizer, batch_size):
    size = len(train_loader.dataset)
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        cur_loss = model(data)["loss"]
        cur_loss.backward()
        optimizer.step()
#         if i % 100 == 0:
#             loss, current = cur_loss.item(), i*len(data)
#             print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        
            

def val(val_loader, model, batch_size):
    val_loss_sum = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_loss_sum += model(data)["loss"]
    avg_val_loss = val_loss_sum / len(val_loader)
    return avg_val_loss
            

### b. Split dataset into train+val (90%) & final_test (10%) parts 

In [40]:
from torch.utils.data import random_split, DataLoader


dataset_1_all = Seq2seq_Dataset(data_1, 1)
dataset_2_all = Seq2seq_Dataset(data_2, 2)

dataset_1_kfold, dataset_1_final_test = random_split(dataset_1_all, [int(len(dataset_1_all) * 0.9), int(len(dataset_1_all) * 0.1)], generator=torch.Generator().manual_seed(42))
dataset_2_kfold, dataset_2_final_test = random_split(dataset_2_all, [int(len(dataset_2_all) * 0.9), int(len(dataset_2_all) * 0.1)], generator=torch.Generator().manual_seed(42))


Finish transforming mapped features' tensors!
Finish transforming unmapped features' tensors!
Finish transforming mapped features' tensors!
Finish transforming unmapped features' tensors!


### c. K-Fold cross validation to train and val model

In [44]:
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
    

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [104]:
kfold_test_loss_1_mean = np.mean(kfold_test_loss_1, axis=1)
kfold_test_loss_1_mean_df = pd.DataFrame(data=kfold_test_loss_1_mean.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_1_mean_df

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,8.101517,7.635693,7.959547,9.442474,8.099477,8.116611,9.297311,9.788095,9.765993,9.19864,9.764281,9.684067,8.797012,9.207513,9.466866


In [107]:
kfold_test_loss_2_mean = np.mean(kfold_test_loss_2, axis=1)
kfold_test_loss_2_mean_df = pd.DataFrame(data=kfold_test_loss_2_mean.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_2_mean_df

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,9.259842,9.859417,8.137906,9.765788,9.47144,7.583542,9.328816,9.597952,9.267031,8.16348,8.809281,8.174178,9.876471,8.020665,9.889639


### d. Trained the final model using all the 90% training data (data used in the above 5-fold cross-validation process)

In [90]:
final_train_dataloader_1 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_param/EHR1/umpf_"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0


/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [91]:
final_train_dataloader_2 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_param/EHR2/umpf_"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


### e. Test the final trained model's performance on 2 EHRs  
* using dataset:  dataset_1_final_test & dataset_2_final_test

In [92]:
models_list_ump_EHR1 = []
models_list_ump_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./each_umpf_model_param/EHR1/umpf_" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./each_umpf_model_param/EHR2/umpf_" + str(i) + "_EHR2.pth"
    
    cur_model_1 = Seq2seq_model(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = Seq2seq_model(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    models_list_ump_EHR1.append(cur_model_1)
    models_list_ump_EHR2.append(cur_model_2)

In [57]:
dataloader_1_test = DataLoader(dataset_1_final_test, batch_size=10)
dataloader_2_test = DataLoader(dataset_2_final_test, batch_size=10)

In [67]:
len_dl_1 = len(dataloader_1_test)
len_dl_2 = len(dataloader_2_test)


In [125]:
patients_test_1 = len(dataloader_1_test.dataset)
patients_test_2 = len(dataloader_2_test.dataset)


# Compute [num_of_ump, num_of_mp*sel_len] matrix for both EHR1 and EHR2, with each row calculated as the average predicted matched features' values for current unmapped feature.


In [128]:
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1 = np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = models_list_ump_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 120)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy(), axis=0) # [120]
        ump_mp_matrix_1[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = models_list_ump_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy(), axis=0)
        ump_mp_matrix_2[ump_id_2][:] += cur_pred_map_batch_sum
ump_mp_matrix_1 = ump_mp_matrix_1 / patients_test_1
ump_mp_matrix_2 = ump_mp_matrix_2 / patients_test_2


# Get cosine similarity matrix via pairwise.cosine_similarity, and apply Gale-Shapley algorithm based on cosine similarity matrix to get predicted matched pairs

In [129]:
from sklearn.metrics import pairwise
cos_sim_matrix_1_to_2 = pairwise.cosine_similarity(ump_mp_matrix_1, ump_mp_matrix_2)
cos_sim_matrix_2_to_1 = pairwise.cosine_similarity(ump_mp_matrix_2, ump_mp_matrix_1)

In [130]:
def Matching_via_HRM(C_X1_train, C_X2_train, P_x1_O_to_R, num_mapped_axis):  # in this case here the small feature sized database is X1, so we need to treat it as hospital and there will be capacities on it.
    ####### ----------  X1 train ------------- ##########

    true_features_pref_X1_train = {}
    cross_recon_features_pref_X1_train = {}
    capacities_X1_train = {}

    for i in range(C_X1_train.shape[0]):  # C_X1_train.shape[0]: number of unmapped features in dataset_1
        sorted_index = np.argsort(-C_X1_train[i, :])
        sorted_col_index = ["C" + str(sorted_index[v] + 1) for v in range(len(sorted_index))]
        true_features_pref_X1_train["R" + str(i + 1)] = sorted_col_index
        capacities_X1_train["R" + str(i + 1)] = 1

    for j in range(C_X1_train.shape[1]): # C_X1_train.shape[1]:  number of unmapped features in dataset_2
        sorted_index = np.argsort(-C_X1_train[:, j])
        sorted_col_index = ["R" + str(sorted_index[v] + 1) for v in range(len(sorted_index))]
        cross_recon_features_pref_X1_train["C" + str(j + 1)] = sorted_col_index

    game_X1_train = HospitalResident.create_from_dictionaries(cross_recon_features_pref_X1_train,
                                                              true_features_pref_X1_train,
                                                              capacities_X1_train)

    ####### ----------  X2 train ------------- ##########

    true_features_pref_X2_train = {}
    cross_recon_features_pref_X2_train = {}
    capacities_X2_train = {}

    for i in range(C_X2_train.shape[0]):  # C_X2_train.shape[0]: number of unmapped features in dataset_2
        sorted_index = np.argsort(-C_X2_train[i, :])
        sorted_col_index = ["C" + str(sorted_index[v] + 1) for v in range(len(sorted_index))]
        true_features_pref_X2_train["R" + str(i + 1)] = sorted_col_index

    for j in range(C_X2_train.shape[1]):  # C_X2_train.shape[1]: number of unmapped features in dataset_1
        sorted_index = np.argsort(-C_X2_train[:, j])
        sorted_col_index = ["R" + str(sorted_index[v] + 1) for v in range(len(sorted_index))]
        cross_recon_features_pref_X2_train["C" + str(j + 1)] = sorted_col_index
        capacities_X2_train["C" + str(j + 1)] = 1

    # create_from_dictionaries(resident_prefs, hospital_prefs, capacities, clean=False)
    game_X2_train = HospitalResident.create_from_dictionaries(true_features_pref_X2_train,
                                                              cross_recon_features_pref_X2_train,
                                                              capacities_X2_train)

       ######   ------------  Final matching -----------   ##########

    print("\n ------- Matching from X1_train  --------- \n")
    matching_x1_train = game_X1_train.solve()
    print(matching_x1_train)

    print("\n ------- Matching from X2_train  --------- \n")
    matching_x2_train = game_X2_train.solve()
    print(matching_x2_train)
    x1_train_y = [int(str(v[0])[1:]) if v else None for v in matching_x1_train.values()]
    x2_train_y = [int(str(v[0])[1:]) if v else None for v in matching_x2_train.values()]

    # matching matrices
    matching_x1_train_matrix = np.zeros(C_X1_train.shape)
    # shape: [num_unmapped_features_in_d1, num_unmapped_features_in_d2]
    matching_x2_train_matrix = np.zeros(np.transpose(C_X2_train).shape)
    # shape: [num_unmapped_features_in_d1, num_unmapped_features_in_d2]

    for i in range(matching_x1_train_matrix.shape[0]):  # number of unmapped features in d_1
        if x1_train_y[i] is not None:
            matching_x1_train_matrix[i, x1_train_y[i] - 1] = 1  # shape: [# of ump features in d1, # of ump features in d2]
        # unmapped feature i in d_1 and unmapped feature "x1_train_y[i] - 1" in d_2 has a match

    for i in range(matching_x2_train_matrix.shape[0]):  # number of unmapped features in d_1
        if x2_train_y[i] is not None:
            matching_x2_train_matrix[i, x2_train_y[i] - 1] = 1  # shape: [# of ump features in d1, # of ump features in d2]
        # unmapped feature i in d_1 and unmapped feature "x2_train_y[i] - 1" in d_2 has a match
    # getting the number of correct matches that had a match in other database
    num_correct_from_x1 = 0
    num_correct_from_x2 = 0
    for i in range(P_x1_O_to_R.shape[0]):  # number of unmapped features in d_1
        if np.all(P_x1_O_to_R[i] == matching_x1_train_matrix[i]):
            # only when the positions of 0-1 are exactly the same, will this condition be true
            num_correct_from_x1 = num_correct_from_x1 + 1
        if np.all(P_x1_O_to_R[i] == matching_x2_train_matrix[i]):
            num_correct_from_x2 = num_correct_from_x2 + 1

    return num_correct_from_x1, num_correct_from_x2, matching_x1_train_matrix, matching_x2_train_matrix


In [133]:
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2, cos_sim_matrix_2_to_1, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C10], R2: [C5], R3: [C15], R4: [C6], R5: [C8], R6: [C1], R7: [C13], R8: [C3], R9: [C12], R10: [C4], R11: [C2], R12: [C7], R13: [C14], R14: [C11], R15: [C9]}

 ------- Matching from X2_train  --------- 

{C1: [R10], C2: [R5], C3: [R15], C4: [R6], C5: [R8], C6: [R1], C7: [R13], C8: [R3], C9: [R12], C10: [R4], C11: [R2], C12: [R7], C13: [R14], C14: [R11], C15: [R9]}


In [134]:
TP_x1 = 0
FP_x1 = 0
TN_x1 = 0
FN_x1 = 0
for i in range(p.shape[0]):
    for j in range(p.shape[1]):
        if (p[i, j] == 1) & (x1_match_matrix_test[i, j] == 1):
            TP_x1 = TP_x1 + 1
        elif (p[i, j] == 1) & (x1_match_matrix_test[i, j] == 0):
            FN_x1 = FN_x1 + 1
        elif (p[i, j] == 0) & (x1_match_matrix_test[i, j] == 0):
            TN_x1 = TN_x1 + 1
        elif (p[i, j] == 0) & (x1_match_matrix_test[i, j] == 1):
            FP_x1 = FP_x1 + 1

TP_x2 = 0
FP_x2 = 0
TN_x2 = 0
FN_x2 = 0
for i in range(p.shape[0]):
    for j in range(p.shape[1]):
        if (p[i, j] == 1) & (x2_match_matrix_test[i, j] == 1):
            TP_x2 = TP_x2 + 1
        elif (p[i, j] == 1) & (x2_match_matrix_test[i, j] == 0):
            FN_x2 = FN_x2 + 1
        elif (p[i, j] == 0) & (x2_match_matrix_test[i, j] == 0):
            TN_x2 = TN_x2 + 1
        elif (p[i, j] == 0) & (x2_match_matrix_test[i, j] == 1):
            FP_x2 = FP_x2 + 1
F1_fromx1 = (2 * TP_x1) / (2 * TP_x1 + FN_x1 + FP_x1)
F1_fromx2 = (2 * TP_x2) / (2 * TP_x2 + FN_x2 + FP_x2)

print("Sim cor F values ", F1_fromx1, F1_fromx2)

Sim cor F values  0.0 0.0


In [135]:
# Former method to get avg_cos_sim_matrix

# num_ump_f_1 = 15
# num_ump_f_2 = 15
# avg_cos_sim_matrix = np.zeros((num_ump_f_1, num_ump_f_2))
# for ump_i_1 in range(15):
#     cur_model_1 = models_list_ump_EHR1[ump_i_1]
#     cur_model_1.eval()
#     cur_umpf_1_to_all_umpf_2 = np.zeros((len_dl_1, num_ump_f_2))
#     for index, (data1, data2) in enumerate(zip(dataloader_1_test, dataloader_2_test)):
#         pred_map_1 = cur_model_1(data1)["predicts"] 
#         pred_map_1_asarray = pred_map_1.detach().numpy() # (10, 120)
#         for ump_i_2 in range(15):
#             cur_model_2 = models_list_ump_EHR2[ump_i_2]
#             cur_model_2.eval()
#             pred_map_2 = cur_model_2(data2)["predicts"]
#             pred_map_2_asarray = pred_map_2.detach().numpy() # (10, 120)
#             # compute cosine similarity on cur batch
#             cur_cosine_sim = np.sum(pred_map_1_asarray*pred_map_2_asarray, axis=1) / (norm(pred_map_1_asarray, axis=1)*norm(pred_map_2_asarray, axis=1))
#             cur_umpf_1_to_all_umpf_2[index][ump_i_2] = np.average(cur_cosine_sim)
#     avg_cos_sim_matrix[ump_i_1, :] = np.average(cur_umpf_1_to_all_umpf_2, axis=0)

    
# print(avg_cos_sim_matrix)
 

        

# The predicted matched features for each unmapped feature in EHR1

In [136]:
# pred_matched_for_umf1 = avg_cos_sim_matrix.argmax(axis = 1)
# pred_matched_for_umf1

# The actual matched feature for each unmapped feature in EHR1

In [137]:
# actual_matched_for_umf1 = p.argmax(axis=1)
# actual_matched_for_umf1

In [96]:
# predicted_perf = (pred_matched_for_umf1 == actual_matched_for_umf1)

In [138]:
# predicted_perf

# Define model_2: feed the output(the final hidden state of the last layer on each time stamp, with shape: [batch_size, seq_len(24), hidden_dim]) to MLP to predict mapped time series data

In [165]:
class Seq2seq_model_pass_final_hs_each_time(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(Seq2seq_model_pass_final_hs_each_time, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        
        self.target_dim = num_mp_f
        
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.MLP_drop1 = nn.Dropout(p=0.2)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim * 2))
        self.MLP_drop2 = nn.Dropout(p=0.2)
        self.dense3 = nn.Linear(int(self.target_dim * 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = input_data[0]
        # shape: [10, 24, 5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, _ = self.RNN(unmapped_features, self.h_0)
        # output: [10, 24, hidden_dim]
        map_predict = self.dense1(output)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop1(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop2(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        
        
        

# K-fold cross validation on the second model

In [166]:
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_model2_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_model2_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model_pass_final_hs_each_time(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_model2_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model_pass_final_hs_each_time(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_model2_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
   

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


# average validation loss on unmapped features in EHR1 using model2

In [167]:
avg_val_loss_data1 = np.mean(kfold_test_model2_loss_1, axis=1)
avg_val_loss_data2 = np.mean(kfold_test_model2_loss_2, axis=1)
avg_val_loss_umpf_1 = pd.DataFrame(data=avg_val_loss_data1.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
avg_val_loss_umpf_2 = pd.DataFrame(data=avg_val_loss_data2.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
avg_val_loss_umpf_1

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,4.677182,3.803522,4.200935,8.242604,4.604284,5.171813,7.595304,9.782699,9.550829,7.344407,9.569802,9.113071,6.269675,7.402231,8.328141


# average validation loss on unmapped features in EHR1 using model1 

In [157]:
kfold_test_loss_1_mean_df

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,8.101517,7.635693,7.959547,9.442474,8.099477,8.116611,9.297311,9.788095,9.765993,9.19864,9.764281,9.684067,8.797012,9.207513,9.466866


# average validation loss on unmapped features in EHR2 using model2

In [168]:
avg_val_loss_umpf_2

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,7.387987,9.592985,4.461771,9.18687,8.233776,3.936945,7.684729,8.505713,7.277887,4.779131,6.207377,4.652694,9.696338,4.269492,9.885467


# average validation loss on unmapped features in EHR2 using model1

In [108]:
kfold_test_loss_2_mean_df

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,9.259842,9.859417,8.137906,9.765788,9.47144,7.583542,9.328816,9.597952,9.267031,8.16348,8.809281,8.174178,9.876471,8.020665,9.889639


# Train the final model_2 using all above 90% training data

In [110]:
final_train_dataloader_1 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model_pass_final_hs_each_time(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_2_param/EHR1/umpf_"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [111]:
final_train_dataloader_2 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model_pass_final_hs_each_time(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_2_param/EHR2/umpf_"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


# Test final trained model on 10% test data

In [112]:
models_2_list_ump_EHR1 = []
models_2_list_ump_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./each_umpf_model_2_param/EHR1/umpf_" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./each_umpf_model_2_param/EHR2/umpf_" + str(i) + "_EHR2.pth"
    
    cur_model_1 = Seq2seq_model_pass_final_hs_each_time(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = Seq2seq_model_pass_final_hs_each_time(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    models_2_list_ump_EHR1.append(cur_model_1)
    models_2_list_ump_EHR2.append(cur_model_2)

/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


# Compute [num_of_ump, num_of_mp*sel_len] matrix for both EHR1 and EHR2, with each row calculated as the average predicted matched features' values for current unmapped feature.

In [147]:
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1_model_2 = np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2_model_2 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = models_2_list_ump_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 24, 5)
#         print(cur_pred_map.detach().numpy().shape)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0) # [120]
        ump_mp_matrix_1_model_2[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = models_2_list_ump_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0)
        ump_mp_matrix_2_model_2[ump_id_2][:] += cur_pred_map_batch_sum
ump_mp_matrix_1_model_2 = ump_mp_matrix_1_model_2 / patients_test_1
ump_mp_matrix_2_model_2 = ump_mp_matrix_2_model_2 / patients_test_2

cos_sim_matrix_1_to_2_model_2 = pairwise.cosine_similarity(ump_mp_matrix_1_model_2, ump_mp_matrix_2_model_2)
cos_sim_matrix_2_to_1_model_2 = pairwise.cosine_similarity(ump_mp_matrix_2_model_2, ump_mp_matrix_1_model_2)
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2_model_2, cos_sim_matrix_2_to_1_model_2, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C15], R2: [C3], R3: [C11], R4: [C4], R5: [C10], R6: [C12], R7: [C7], R8: [C2], R9: [C1], R10: [C9], R11: [C14], R12: [C8], R13: [C5], R14: [C6], R15: [C13]}

 ------- Matching from X2_train  --------- 

{C1: [R15], C2: [R3], C3: [R11], C4: [R4], C5: [R10], C6: [R12], C7: [R7], C8: [R2], C9: [R1], C10: [R9], C11: [R14], C12: [R8], C13: [R5], C14: [R6], C15: [R13]}


In [148]:
print(correct_with_match_from_x1_test)
print(correct_with_match_from_x2_test)

1
1


In [175]:
def F1_score(p, x1_match_matrix_test, x2_match_matrix_test):
    TP_x1 = 0
    FP_x1 = 0
    TN_x1 = 0
    FN_x1 = 0
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if (p[i, j] == 1) & (x1_match_matrix_test[i, j] == 1):
                TP_x1 = TP_x1 + 1
            elif (p[i, j] == 1) & (x1_match_matrix_test[i, j] == 0):
                FN_x1 = FN_x1 + 1
            elif (p[i, j] == 0) & (x1_match_matrix_test[i, j] == 0):
                TN_x1 = TN_x1 + 1
            elif (p[i, j] == 0) & (x1_match_matrix_test[i, j] == 1):
                FP_x1 = FP_x1 + 1

    TP_x2 = 0
    FP_x2 = 0
    TN_x2 = 0
    FN_x2 = 0
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if (p[i, j] == 1) & (x2_match_matrix_test[i, j] == 1):
                TP_x2 = TP_x2 + 1
            elif (p[i, j] == 1) & (x2_match_matrix_test[i, j] == 0):
                FN_x2 = FN_x2 + 1
            elif (p[i, j] == 0) & (x2_match_matrix_test[i, j] == 0):
                TN_x2 = TN_x2 + 1
            elif (p[i, j] == 0) & (x2_match_matrix_test[i, j] == 1):
                FP_x2 = FP_x2 + 1
    F1_fromx1 = (2 * TP_x1) / (2 * TP_x1 + FN_x1 + FP_x1)
    F1_fromx2 = (2 * TP_x2) / (2 * TP_x2 + FN_x2 + FP_x2)

    print("Sim cor F values ", F1_fromx1, F1_fromx2)

In [144]:
F1_score(p, x1_match_matrix_test, x2_match_matrix_test)

Sim cor F values  0.06666666666666667 0.06666666666666667


# Ablation study 1: Add more RNN layers(tested on 3 stacking layers first) based on model2's structure

In [159]:
class ablation_model_more_rnn_layers(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(ablation_model_more_rnn_layers, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(3, self.batch_size, hidden_dim) # 3 is the number of stacked layers
        self.RNN = nn.RNN(input_dim, hidden_dim, num_layers=3, batch_first=True, dropout=0.2)
        
        self.target_dim = num_mp_f
        
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.MLP_drop1 = nn.Dropout(p=0.2)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim * 2))
        self.MLP_drop2 = nn.Dropout(p=0.2)
        self.dense3 = nn.Linear(int(self.target_dim * 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = input_data[0]
        # shape: [10, 24, 5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, _ = self.RNN(unmapped_features, self.h_0)
        # output: [10, 24, hidden_dim]
        map_predict = self.dense1(output)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop1(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop2(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        
        
        

In [162]:
# 5-fold cross-validation model
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_more_rnn_layers(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_more_rnn_layers(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
    

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [163]:
kfold_test_loss_1_mean_abla_model = np.mean(kfold_test_loss_1, axis=1)
kfold_test_loss_1_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_1_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_1_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,4.567791,3.56303,4.688455,8.337237,5.214758,4.984651,7.759979,9.783238,9.567906,7.515486,9.561841,9.157829,6.315001,7.505744,8.340211


In [164]:
kfold_test_loss_2_mean_abla_model = np.mean(kfold_test_loss_2, axis=1)
kfold_test_loss_2_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_2_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_2_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,7.51338,9.627464,4.565676,9.239512,8.309349,3.907383,7.682642,8.379988,7.320166,4.776638,6.406842,4.906768,9.710076,4.313153,9.883927


In [169]:
# train final model using all the 90% dataset
final_train_dataloader_1_abla_m1 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_more_rnn_layers(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1_abla_m1, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_1/EHR1/"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [170]:
# train final model using all the 90% dataset
final_train_dataloader_2_abla_m1 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_more_rnn_layers(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2_abla_m1, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_1/EHR2/"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [171]:
# load trained models
# models_2_list_ump_EHR1 = []
# models_2_list_ump_EHR2 = []
abla_1_models_list_EHR1 = []
abla_1_models_list_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./ablation_model_params/Ablation_model_1/EHR1/" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./ablation_model_params/Ablation_model_1/EHR2/" + str(i) + "_EHR2.pth"
    
    cur_model_1 = ablation_model_more_rnn_layers(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = ablation_model_more_rnn_layers(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    abla_1_models_list_EHR1.append(cur_model_1)
    abla_1_models_list_EHR2.append(cur_model_2)
    
# Get the avg_cos_sim_matrix based on ump_map_matrix using the predicted mapped features' values from the ump features
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1_abla_1= np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2_abla_1 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = abla_1_models_list_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 24, 5)
#         print(cur_pred_map.detach().numpy().shape)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0) # [120]
        ump_mp_matrix_1_abla_1[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = abla_1_models_list_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0)
        ump_mp_matrix_2_abla_1[ump_id_2][:] += cur_pred_map_batch_sum

ump_mp_matrix_1_abla_1 = ump_mp_matrix_1_abla_1 / patients_test_1
ump_mp_matrix_2_abla_1 = ump_mp_matrix_2_abla_1 / patients_test_2

cos_sim_matrix_1_to_2_abla_1 = pairwise.cosine_similarity(ump_mp_matrix_1_abla_1, ump_mp_matrix_2_abla_1)
cos_sim_matrix_2_to_1_abla_1 = pairwise.cosine_similarity(ump_mp_matrix_2_abla_1, ump_mp_matrix_1_abla_1)
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2_abla_1, cos_sim_matrix_2_to_1_abla_1, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C4], R2: [C10], R3: [C8], R4: [C3], R5: [C12], R6: [C2], R7: [C7], R8: [C15], R9: [C14], R10: [C1], R11: [C6], R12: [C11], R13: [C5], R14: [C13], R15: [C9]}

 ------- Matching from X2_train  --------- 

{C1: [R4], C2: [R10], C3: [R8], C4: [R3], C5: [R12], C6: [R2], C7: [R7], C8: [R15], C9: [R14], C10: [R1], C11: [R6], C12: [R11], C13: [R5], C14: [R13], C15: [R9]}


In [174]:
F1_score(p, x1_match_matrix_test, x2_match_matrix_test)

Sim cor F values  0.26666666666666666 0.26666666666666666


# Ablation study 2: Only add one Dropout layer after the third linear layer (the layer before the last linear layer)

In [176]:
class ablation_model_only_one_dp(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(ablation_model_only_one_dp, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        
        self.target_dim = num_mp_f
        
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim * 2))
        self.dense3 = nn.Linear(int(self.target_dim * 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = input_data[0]
        # shape: [10, 24, 5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, _ = self.RNN(unmapped_features, self.h_0)
        # output: [10, 24, hidden_dim]
        map_predict = self.dense1(output)
        map_predict = nn.ReLU()(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}

In [177]:
# 5-fold cross-validation model
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_only_one_dp(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_only_one_dp(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
    

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [178]:
kfold_test_loss_1_mean_abla_model = np.mean(kfold_test_loss_1, axis=1)
kfold_test_loss_1_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_1_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_1_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,4.46891,3.812273,5.600725,8.156265,4.976056,4.797884,7.573601,9.782285,9.550715,7.408799,9.547081,9.119084,6.243166,7.425105,8.414688


In [179]:
kfold_test_loss_2_mean_abla_model = np.mean(kfold_test_loss_2, axis=1)
kfold_test_loss_2_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_2_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_2_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,7.606606,9.608952,4.685113,9.179137,8.446557,4.305447,7.702867,8.406574,7.365686,4.755599,6.157526,5.095134,9.690174,4.667945,9.88554


In [180]:
# train the final model using all 90% dataset
final_train_dataloader_1_abla_m2 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_only_one_dp(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1_abla_m2, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_2/EHR1/"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [181]:
# train the final model using all 90% dataset
final_train_dataloader_2_abla_m2 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_only_one_dp(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2_abla_m2, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_2/EHR2/"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [182]:
abla_2_models_list_EHR1 = []
abla_2_models_list_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./ablation_model_params/Ablation_model_2/EHR1/" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./ablation_model_params/Ablation_model_2/EHR2/" + str(i) + "_EHR2.pth"
    
    cur_model_1 = ablation_model_only_one_dp(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = ablation_model_only_one_dp(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    abla_2_models_list_EHR1.append(cur_model_1)
    abla_2_models_list_EHR2.append(cur_model_2)
    
# Get the avg_cos_sim_matrix based on ump_map_matrix using the predicted mapped features' values from the ump features
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1_abla_2= np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2_abla_2 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = abla_2_models_list_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 24, 5)
#         print(cur_pred_map.detach().numpy().shape)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0) # [120]
        ump_mp_matrix_1_abla_2[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = abla_2_models_list_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0)
        ump_mp_matrix_2_abla_2[ump_id_2][:] += cur_pred_map_batch_sum

ump_mp_matrix_1_abla_2 = ump_mp_matrix_1_abla_2 / patients_test_1
ump_mp_matrix_2_abla_2 = ump_mp_matrix_2_abla_2 / patients_test_2

cos_sim_matrix_1_to_2_abla_2 = pairwise.cosine_similarity(ump_mp_matrix_1_abla_2, ump_mp_matrix_2_abla_2)
cos_sim_matrix_2_to_1_abla_2 = pairwise.cosine_similarity(ump_mp_matrix_2_abla_2, ump_mp_matrix_1_abla_2)
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2_abla_2, cos_sim_matrix_2_to_1_abla_2, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C12], R2: [C10], R3: [C9], R4: [C8], R5: [C5], R6: [C6], R7: [C7], R8: [C15], R9: [C2], R10: [C1], R11: [C4], R12: [C13], R13: [C11], R14: [C3], R15: [C14]}

 ------- Matching from X2_train  --------- 

{C1: [R12], C2: [R10], C3: [R9], C4: [R8], C5: [R5], C6: [R6], C7: [R7], C8: [R15], C9: [R2], C10: [R1], C11: [R4], C12: [R13], C13: [R11], C14: [R3], C15: [R14]}


In [183]:
F1_score(p, x1_match_matrix_test, x2_match_matrix_test)

Sim cor F values  0.3333333333333333 0.3333333333333333


# Ablation study 3: Delete all the Dropout layers (only keep ReLU activation function) based on model2's structure

In [184]:
class ablation_model_no_dp_layers(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(ablation_model_no_dp_layers, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        
        self.target_dim = num_mp_f
        
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim * 2))
        self.dense3 = nn.Linear(int(self.target_dim * 2), self.target_dim)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = input_data[0]
        # shape: [10, 24, 5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, _ = self.RNN(unmapped_features, self.h_0)
        # output: [10, 24, hidden_dim]
        map_predict = self.dense1(output)
        map_predict = nn.ReLU()(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        

In [185]:
# 5fold cross validation on abla model 3
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_no_dp_layers(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = ablation_model_no_dp_layers(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
    

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [186]:
kfold_test_loss_1_mean_abla_model = np.mean(kfold_test_loss_1, axis=1)
kfold_test_loss_1_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_1_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_1_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,4.599545,3.976057,5.182454,8.283076,4.997666,4.461489,8.064072,9.782189,9.542177,7.529328,9.538702,9.077019,6.3282,7.452664,8.342861


In [187]:
kfold_test_loss_2_mean_abla_model = np.mean(kfold_test_loss_2, axis=1)
kfold_test_loss_2_mean_df_abla_model = pd.DataFrame(data=kfold_test_loss_2_mean_abla_model.reshape(1, 15), columns=["umpf"+str(i) for i in range(1, 16)])
kfold_test_loss_2_mean_df_abla_model

,umpf1,umpf2,umpf3,umpf4,umpf5,umpf6,umpf7,umpf8,umpf9,umpf10,umpf11,umpf12,umpf13,umpf14,umpf15
0,7.346642,9.604694,4.308839,9.16393,8.26945,3.350706,7.571636,8.357831,7.291438,5.168404,6.673473,4.971489,9.702716,4.60717,9.884546


In [188]:
# train the final model using all 90% dataset: EHR1
final_train_dataloader_1_abla_m3 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_no_dp_layers(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1_abla_m3, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_3/EHR1/"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [189]:
# train the final model using all 90% dataset: EHR2
final_train_dataloader_2_abla_m3 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_no_dp_layers(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2_abla_m3, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_3/EHR2/"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [190]:
# test trained model's performance on left 10% data
abla_3_models_list_EHR1 = []
abla_3_models_list_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./ablation_model_params/Ablation_model_3/EHR1/" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./ablation_model_params/Ablation_model_3/EHR2/" + str(i) + "_EHR2.pth"
    
    cur_model_1 = ablation_model_no_dp_layers(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = ablation_model_no_dp_layers(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    abla_3_models_list_EHR1.append(cur_model_1)
    abla_3_models_list_EHR2.append(cur_model_2)
    
# Get the avg_cos_sim_matrix based on ump_map_matrix using the predicted mapped features' values from the ump features
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1_abla_3= np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2_abla_3 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = abla_3_models_list_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 24, 5)
#         print(cur_pred_map.detach().numpy().shape)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0) # [120]
        ump_mp_matrix_1_abla_3[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = abla_3_models_list_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0)
        ump_mp_matrix_2_abla_3[ump_id_2][:] += cur_pred_map_batch_sum

ump_mp_matrix_1_abla_3 = ump_mp_matrix_1_abla_3 / patients_test_1
ump_mp_matrix_2_abla_3 = ump_mp_matrix_2_abla_3 / patients_test_2

cos_sim_matrix_1_to_2_abla_3 = pairwise.cosine_similarity(ump_mp_matrix_1_abla_3, ump_mp_matrix_2_abla_3)
cos_sim_matrix_2_to_1_abla_3 = pairwise.cosine_similarity(ump_mp_matrix_2_abla_3, ump_mp_matrix_1_abla_3)
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2_abla_3, cos_sim_matrix_2_to_1_abla_3, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C13], R2: [C9], R3: [C15], R4: [C10], R5: [C12], R6: [C14], R7: [C6], R8: [C1], R9: [C11], R10: [C3], R11: [C5], R12: [C2], R13: [C4], R14: [C8], R15: [C7]}

 ------- Matching from X2_train  --------- 

{C1: [R13], C2: [R9], C3: [R15], C4: [R10], C5: [R12], C6: [R14], C7: [R6], C8: [R1], C9: [R11], C10: [R3], C11: [R5], C12: [R2], C13: [R4], C14: [R8], C15: [R7]}


In [191]:
F1_score(p, x1_match_matrix_test, x2_match_matrix_test)

Sim cor F values  0.06666666666666667 0.06666666666666667


# Ablation study 4: change the activation function in the MLP part from ReLU to LeakyReLU

In [194]:
class ablation_model_leakyReLU(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(ablation_model_leakyReLU, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        
        self.target_dim = num_mp_f
        
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.MLP_drop1 = nn.Dropout(p=0.2)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim * 2))
        self.MLP_drop2 = nn.Dropout(p=0.2)
        self.dense3 = nn.Linear(int(self.target_dim * 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = input_data[0]
        # shape: [10, 24, 5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, _ = self.RNN(unmapped_features, self.h_0)
        # output: [10, 24, hidden_dim]
        map_predict = self.dense1(output)
        map_predict = nn.LeakyReLU(0.1)(map_predict)
        map_predict = self.MLP_drop1(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.LeakyReLU(0.1)(map_predict)
        map_predict = self.MLP_drop2(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.LeakyReLU(0.1)(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        
        
        

In [195]:
# train the final model using all 90% dataset: EHR1
final_train_dataloader_1_abla_m4 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_leakyReLU(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1_abla_m4, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_4/EHR1/"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [196]:
# train the final model using all 90% dataset: EHR1
final_train_dataloader_2_abla_m4 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = ablation_model_leakyReLU(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2_abla_m4, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./ablation_model_params/Ablation_model_4/EHR2/"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [197]:
# test trained model's performance on left 10% data
abla_4_models_list_EHR1 = []
abla_4_models_list_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./ablation_model_params/Ablation_model_4/EHR1/" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./ablation_model_params/Ablation_model_4/EHR2/" + str(i) + "_EHR2.pth"
    
    cur_model_1 = ablation_model_leakyReLU(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = ablation_model_leakyReLU(i-1, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    abla_4_models_list_EHR1.append(cur_model_1)
    abla_4_models_list_EHR2.append(cur_model_2)
    
# Get the avg_cos_sim_matrix based on ump_map_matrix using the predicted mapped features' values from the ump features
num_ump_f_1 = 15
num_ump_f_2 = 15
num_mp_f = 5
seq_len = 24
ump_mp_matrix_1_abla_4= np.zeros((num_ump_f_1, seq_len * num_mp_f))
ump_mp_matrix_2_abla_4 = np.zeros((num_ump_f_2, seq_len * num_mp_f))
for ump_id_1 in range(num_ump_f_1):
    cur_model_1 = abla_4_models_list_EHR1[ump_id_1]
    cur_model_1.eval()
    for data in dataloader_1_test:
        cur_pred_map = cur_model_1(data)["predicts"] # (10, 24, 5)
#         print(cur_pred_map.detach().numpy().shape)
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0) # [120]
        ump_mp_matrix_1_abla_4[ump_id_1][:] += cur_pred_map_batch_sum

for ump_id_2 in range(num_ump_f_2):
    cur_model_2 = abla_4_models_list_EHR2[ump_id_2]
    cur_model_2.eval()
    for data in dataloader_2_test:
        cur_pred_map = cur_model_2(data)["predicts"]
        cur_pred_map_batch_sum = np.sum(cur_pred_map.detach().numpy().reshape((10, 120)), axis=0)
        ump_mp_matrix_2_abla_4[ump_id_2][:] += cur_pred_map_batch_sum

ump_mp_matrix_1_abla_4 = ump_mp_matrix_1_abla_4 / patients_test_1
ump_mp_matrix_2_abla_4 = ump_mp_matrix_2_abla_4 / patients_test_2

cos_sim_matrix_1_to_2_abla_4 = pairwise.cosine_similarity(ump_mp_matrix_1_abla_4, ump_mp_matrix_2_abla_4)
cos_sim_matrix_2_to_1_abla_4 = pairwise.cosine_similarity(ump_mp_matrix_2_abla_4, ump_mp_matrix_1_abla_4)
correct_with_match_from_x1_test, correct_with_match_from_x2_test, x1_match_matrix_test, x2_match_matrix_test = Matching_via_HRM(cos_sim_matrix_1_to_2_abla_4, cos_sim_matrix_2_to_1_abla_4, p, 5)



 ------- Matching from X1_train  --------- 

{R1: [C15], R2: [C14], R3: [C1], R4: [C4], R5: [C11], R6: [C10], R7: [C7], R8: [C6], R9: [C8], R10: [C12], R11: [C13], R12: [C9], R13: [C5], R14: [C3], R15: [C2]}

 ------- Matching from X2_train  --------- 

{C1: [R15], C2: [R14], C3: [R1], C4: [R4], C5: [R11], C6: [R10], C7: [R7], C8: [R6], C9: [R8], C10: [R12], C11: [R13], C12: [R9], C13: [R5], C14: [R3], C15: [R2]}


In [199]:
correct_with_match_from_x1_test

3

In [198]:
F1_score(p, x1_match_matrix_test, x2_match_matrix_test)

Sim cor F values  0.2 0.2
